In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import plotly.graph_objects as go

In [165]:
clinical_data = pd.read_csv('/gladstone/finkbeiner/steve/work/data/npsad_data/monika/Amy_plaque_Results/clinical_metadata.csv')
plaque_counts = pd.read_csv("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/csv_merged_data/final_quantify_internal_dataset.csv")

In [ ]:
plaque_counts.head(2)

,Unnamed: 0,image_name,total_core_plaques,total_diffuse_plaques,total_caa_plaques,avg_area,avg_diameter,avg_eccentricity,age,sex,ethinicity,Dx,total_plaques
0,0,XE09-041_1_AmyB_1,1,1,34,43693.305556,231.350093,0.766523,68.0,Male,Hispanic,Harry's plaintext: AD-Def. | CERAD_1: Definite...,36
1,1,XE14-004_1_AmyB_1,111,30,2,47327.440559,231.262148,0.661020,86.0,Female,White,Harry's plaintext: AD-Def. | CERAD_1: Probable...,143


In [4]:
len(plaque_counts)

32

In [166]:
plaque_counts["Imaging_XENum"]=plaque_counts["image_name"].apply(lambda l: l.split("_1_")[0])

In [167]:
plaque_counts_merged = pd.merge(plaque_counts,clinical_data[["Dx","Imaging_XENum","Plq_Mn","Plq_Mn_MFG","MidPlaquesValue","MidPlaquesWCoresValue","CDRScore"]], on="Imaging_XENum", how = "left")

In [168]:
plaque_counts_merged.head(2)

,Unnamed: 0,image_name,total_core_plaques,total_diffuse_plaques,total_caa_plaques,avg_area,avg_diameter,avg_eccentricity,age,sex,ethinicity,Dx_x,total_plaques,Imaging_XENum,Dx_y,Plq_Mn,Plq_Mn_MFG,MidPlaquesValue,MidPlaquesWCoresValue,CDRScore
0,0,XE09-041_1_AmyB_1,1,1,34,43693.305556,231.350093,0.766523,68.0,Male,Hispanic,Harry's plaintext: AD-Def. | CERAD_1: Definite...,36,XE09-041,Harry's plaintext: AD-Def. | CERAD_1: Definite...,8.56,8.66,3.0,3.0,1.0
1,1,XE14-004_1_AmyB_1,111,30,2,47327.440559,231.262148,0.661020,86.0,Female,White,Harry's plaintext: AD-Def. | CERAD_1: Probable...,143,XE14-004,Harry's plaintext: AD-Def. | CERAD_1: Probable...,9.22,8.41,3.0,3.0,3.0


In [8]:
plaque_counts_merged["Dx_x"].unique()

array(["Harry's plaintext: AD-Def. | CERAD_1: Definite Alzheimer's disease | CERAD_2: ",
       "Harry's plaintext: AD-Def. | CERAD_1: Probable Alzheimer's disease | CERAD_2: Others: other",
       "Harry's plaintext: AD-Def. | CERAD_1: Probable Alzheimer's disease | CERAD_2: ",
       "Harry's plaintext: AD-Def. | CERAD_1: Definite Alzheimer's disease | CERAD_2: Vascular disease: combined lacunar and large infractions",
       "Harry's plaintext: AD-Def. | CERAD_1: Definite Alzheimer's disease | CERAD_2: Vascular disease: infarct(s) only",
       "Harry's plaintext: AD-Def. | CERAD_1: Probable Alzheimer's disease | CERAD_2: Vascular disease: combined lacunar and large infractions",
       "Harry's plaintext: AD-Def. | CERAD_1: Others: other | CERAD_2: Probable Alzheimer's disease",
       "Harry's plaintext: AD-Def. | CERAD_1: Definite Alzheimer's disease | CERAD_2: Vascular disease: lacunar state only",
       "Harry's plaintext: AD-Def. | CERAD_1: Probable Alzheimer's disease | CERA

In [169]:
plaque_counts_merged["CDRScore"].unique()

array([1. , 3. , 0.5, 5. , 4. , 2. , nan, 0. ])

In [183]:
plaque_counts_merged["Cerad_1"] =  plaque_counts_merged["Dx_y"].apply(lambda l: l.split("|")[1].split(":")[1])

In [184]:
plaque_counts_merged["Cerad_1_score"] =   plaque_counts_merged["Cerad_1"].apply(lambda l: 3 if l.strip()=="Probable Alzheimer's disease" else 4 if l.strip()=="Definite Alzheimer's disease" else 0)

In [185]:
plaque_counts_merged["Cerad_1_score"].value_counts()

4    24
3     7
0     1
Name: Cerad_1_score, dtype: int64

In [186]:
plaque_counts_merged["Cerad_1_score"].describe()

count    32.000000
mean      3.656250
std       0.787375
min       0.000000
25%       3.750000
50%       4.000000
75%       4.000000
max       4.000000
Name: Cerad_1_score, dtype: float64

In [163]:
def plotly_box_plot(data, cerad_dict, title, x_axis_title, y_axis_title):
  fig = go.Figure()
  for i in range(3):
    fig.add_trace(go.Box(y=data[i],
                boxpoints='all', # can also be outliers, or suspectedoutliers, or False
                jitter=0.3, # add some jitter for a better separation between points
                pointpos=0, # relative position of points wrt box
                name =cerad_dict[i]))
  fig.update_layout(height=900, width = 1000)
  fig.update_layout(
    xaxis_title_text=x_axis_title, # xaxis label
    yaxis_title_text=y_axis_title) # yaxis label
  fig.update_layout(title=title)
  fig.update_layout( plot_bgcolor='white')
  fig.update_xaxes( mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
  fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)

  fig.show()

In [189]:
def boxplot_plaque(var, title, x_axis_title, y_axis_title):
    def_count = plaque_counts_merged[plaque_counts_merged["Cerad_1_score"]==4][var].values
    prob_count = plaque_counts_merged[plaque_counts_merged["Cerad_1_score"]==3][var].values
    othrs_count = plaque_counts_merged[plaque_counts_merged["Cerad_1_score"]==0][var].values
    data = np.array([def_count,prob_count,othrs_count ])
    cerad_list = ["4 - Definite Alzheimer's disease","3 - Probable Alzheimer's disease","0  - Others"]
    plotly_box_plot(data, cerad_list, title,x_axis_title, y_axis_title)
    
    _, p = stats.ttest_ind(def_count, prob_count, nan_policy='omit')
    print("p-value for def and prob", p)
    #_, p = stats.ttest_ind(prob_count, othrs_count, nan_policy='omit')
    #print("p-value for prob and others", p)
    #_, p = stats.ttest_ind(prob_count, othrs_count, nan_policy='omit')
    #print("p-value for prob and others", p)
 
title =  "Model predicted - total plaque count vs CERAD Score" 
y_axis_title = "Model Predicted - Total Plaque Count"  
x_axis_title = 'CERAD Score'
boxplot_plaque("total_plaques", title, x_axis_title, y_axis_title )

/tmp/ipykernel_38967/2418987200.py:5: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



p-value for def and prob 0.03823889736086657


In [191]:
title =  "Model predicted - Cored plaque count vs CERAD Score" 
y_axis_title = "Model Predicted - Cored plaque count"  
x_axis_title = 'CERAD Score'
boxplot_plaque("total_core_plaques", title,x_axis_title, y_axis_title )

/tmp/ipykernel_38967/2418987200.py:5: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



p-value for def and prob 0.035877734030958765


In [192]:
title =  "Model predicted - Diffuse plaque count vs CERAD Score" 
y_axis_title = "Model Predicted - Diffuse plaque count"  
boxplot_plaque("total_diffuse_plaques", title, x_axis_title, y_axis_title )

/tmp/ipykernel_38967/2418987200.py:5: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



p-value for def and prob 0.17572369312951602


In [193]:
title =  "Model predicted - CAA plaque count vs CERAD Score" 
y_axis_title = "Model Predicted - CAA plaque count"  
boxplot_plaque("total_caa_plaques", title,x_axis_title, y_axis_title )

/tmp/ipykernel_38967/2418987200.py:5: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



p-value for def and prob 0.08650357184947381


## Spearman Corr

In [87]:
from scipy import stats

In [88]:
def calculate_spearman_coeff(x, y):
    res = stats.spearmanr(x, y)
    #res_exact = stats.permutation_test((x,), res,
    #                               permutation_type='pairings')
    return res
#res_exact.p_value

In [89]:
plaque_counts_merged.columns

Index(['Unnamed: 0', 'image_name', 'total_core_plaques',
       'total_diffuse_plaques', 'total_caa_plaques', 'avg_area',
       'avg_diameter', 'avg_eccentricity', 'age', 'sex', 'ethinicity', 'Dx_x',
       'total_plaques', 'Imaging_XENum', 'Dx_y', 'Plq_Mn', 'Plq_Mn_MFG',
       'MidPlaquesValue', 'MidPlaquesWCoresValue', 'Cerad_1', 'Cerad_1_score'],
      dtype='object')

In [197]:
x = plaque_counts_merged["total_plaques"]
y = plaque_counts_merged["Plq_Mn"]
calculate_spearman_coeff(x, y)

SpearmanrResult(correlation=0.656980477597567, pvalue=4.419188669906899e-05)

In [91]:
x = plaque_counts_merged["total_core_plaques"]
y = plaque_counts_merged["Plq_Mn"]
calculate_spearman_coeff(x, y)

SpearmanrResult(correlation=0.7810380132884244, pvalue=1.3317196601918832e-07)

In [92]:
x = plaque_counts_merged["total_plaques"]
y = plaque_counts_merged["Plq_Mn_MFG"]
calculate_spearman_coeff(x, y)

SpearmanrResult(correlation=0.5133846718005134, pvalue=0.0026555504230795046)

In [93]:
x = plaque_counts_merged["total_core_plaques"]
y = plaque_counts_merged["Plq_Mn_MFG"]
calculate_spearman_coeff(x, y)

SpearmanrResult(correlation=0.6897753585180265, pvalue=1.2581335704425582e-05)

In [94]:
x = plaque_counts_merged["total_plaques"]
y = plaque_counts_merged["MidPlaquesValue"]
calculate_spearman_coeff(x, y)

SpearmanrResult(correlation=0.5241329788859538, pvalue=0.0020763475078348346)

In [95]:
x = plaque_counts_merged["total_core_plaques"]
y = plaque_counts_merged["MidPlaquesValue"]
calculate_spearman_coeff(x, y)

SpearmanrResult(correlation=0.6312703743574228, pvalue=0.00010701723110368686)

In [96]:
x = plaque_counts_merged["total_plaques"]
y = plaque_counts_merged["MidPlaquesWCoresValue"]
calculate_spearman_coeff(x, y)

SpearmanrResult(correlation=0.5241329788859538, pvalue=0.0020763475078348346)

In [97]:
x = plaque_counts_merged["total_core_plaques"]
y = plaque_counts_merged["MidPlaquesWCoresValue"]
calculate_spearman_coeff(x, y)

SpearmanrResult(correlation=0.6312703743574228, pvalue=0.00010701723110368686)

In [98]:
x = plaque_counts_merged["total_plaques"]
y = plaque_counts_merged["age"]
calculate_spearman_coeff(x, y)

SpearmanrResult(correlation=-0.5433014056357187, pvalue=0.0013117648620232408)

## Correlation Plots

In [67]:
fields = ["Plq_Mn", "Plq_Mn_MFG","MidPlaquesValue","MidPlaquesWCoresValue"]
for f in fields:
    print(f)
    x = plaque_counts_merged.groupby("Cerad_1")[f].agg({"min","max","mean"})
    print(x)
    print("----------------")

Plq_Mn
                                     mean    max   min
Cerad_1                                               
 Definite Alzheimer's disease   15.667500  35.06  6.21
 Others                          4.050000   4.05  4.05
 Probable Alzheimer's disease    6.101429   9.22  2.05
----------------
Plq_Mn_MFG
                                     mean    max   min
Cerad_1                                               
 Definite Alzheimer's disease   16.241667  40.51  5.10
 Others                          4.870000   4.87  4.87
 Probable Alzheimer's disease    5.721429   8.41  2.56
----------------
MidPlaquesValue
                                    mean  max  min
Cerad_1                                           
 Definite Alzheimer's disease   4.583333  5.0  3.0
 Others                         1.000000  1.0  1.0
 Probable Alzheimer's disease   2.714286  3.0  1.0
----------------
MidPlaquesWCoresValue
                                    mean  max  min
Cerad_1                              

In [107]:
fields = ["total_plaques", "total_core_plaques","total_diffuse_plaques","total_caa_plaques"]
for f in fields:
    print(f)
    x = plaque_counts_merged.groupby("Cerad_1")[f].agg({"min","max","mean","median"})
    print(x)
    print("----------------")

total_plaques
                                      mean   max  median  min
Cerad_1                                                      
 Definite Alzheimer's disease   564.875000  1587   470.5   36
 Others                          25.000000    25    25.0   25
 Probable Alzheimer's disease   194.714286   713   143.0    5
----------------
total_core_plaques
                                      mean  max  median  min
Cerad_1                                                     
 Definite Alzheimer's disease   228.166667  757   163.5    1
 Others                           9.000000    9     9.0    9
 Probable Alzheimer's disease    57.000000  119    63.0    1
----------------
total_diffuse_plaques
                                      mean  max  median  min
Cerad_1                                                     
 Definite Alzheimer's disease   294.000000  984   202.0    1
 Others                          15.000000   15    15.0   15
 Probable Alzheimer's disease   132.428571  623    3

In [194]:
import plotly.express as px
import statsmodels.api as sm

df = plaque_counts_merged[plaque_counts_merged["Cerad_1_score"]!=0]
df["CERAD Score"] = df["Cerad_1_score"].astype(str) + "-" + df["Cerad_1"]
val_y = "Plq_Mn"
val_x= "total_plaques"
title = "Total plaque count vs Plq_Mn (Average density of neuritic plaques)"


fig = px.scatter(df, x=val_x, y=val_y,  trendline="ols", color="CERAD Score", title=title, trendline_scope="overall", trendline_color_override="black")
#trendline="ols",
fig.update_layout(height = 700, width = 900)
fig.update_layout( plot_bgcolor='white')
fig.update_xaxes( mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig.add_annotation(x=1000, y=21,
            text="R-squared : 0.773",
            showarrow=True,
            arrowhead=1,
            font=dict(
                color="green",
                size=12
            ),arrowcolor="green")
fig.update_layout(
    xaxis_title_text='Total Plaque Count', # xaxis label
    yaxis_title_text="Plq_Mn (Average density of neuritic plaques)") # yaxis label
fig.show()

#results = px.get_trendline_results(fig)
#print(results.sumaary())

output = sm.OLS(df[val_y], df[val_x]).fit() 
print(output.summary())

/tmp/ipykernel_38967/2132785826.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



                                 OLS Regression Results                                
Dep. Variable:                 Plq_Mn   R-squared (uncentered):                   0.774
Model:                            OLS   Adj. R-squared (uncentered):              0.766
Method:                 Least Squares   F-statistic:                              102.5
Date:                Mon, 08 May 2023   Prob (F-statistic):                    3.42e-11
Time:                        15:45:11   Log-Likelihood:                         -106.28
No. Observations:                  31   AIC:                                      214.6
Df Residuals:                      30   BIC:                                      216.0
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------

In [200]:
df = plaque_counts_merged[plaque_counts_merged["Cerad_1_score"]!=0]
df["CERAD Score"] = df["Cerad_1_score"].astype(str) + "-" + df["Cerad_1"]
val_y = "Plq_Mn"
val_x= "total_core_plaques"
title = "Cored plaque count vs Plq_Mn (Average density of neuritic plaques)"
fig = px.scatter(df, x=val_x, y=val_y, color="CERAD Score", trendline="ols", title=title, trendline_scope="overall",trendline_color_override="black")
#trendline="ols",
fig.update_layout(height = 700, width = 900)
fig.update_layout( plot_bgcolor='white')
fig.update_xaxes( mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig.add_annotation(x=400, y=20,
            text="R-squared : 0.744",
            showarrow=True,
            arrowhead=1,
            font=dict(
                color="green",
                size=12
            ),arrowcolor="green")
fig.update_layout(
    xaxis_title_text='Cored Plaque Count', # xaxis label
    yaxis_title_text="Plq_Mn (Average density of neuritic plaques)") # yaxis label
fig.show()


#results = px.get_trendline_results(fig)
#print(results.sumaary())

output = sm.OLS(df[val_y], df[val_x]).fit() 
print(output.summary())

/tmp/ipykernel_38967/1707711366.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



                                 OLS Regression Results                                
Dep. Variable:                 Plq_Mn   R-squared (uncentered):                   0.744
Model:                            OLS   Adj. R-squared (uncentered):              0.736
Method:                 Least Squares   F-statistic:                              87.38
Date:                Mon, 08 May 2023   Prob (F-statistic):                    2.15e-10
Time:                        16:59:04   Log-Likelihood:                         -108.17
No. Observations:                  31   AIC:                                      218.3
Df Residuals:                      30   BIC:                                      219.8
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------

## Correlation with Age

In [80]:
df = plaque_counts_merged
val_y = "total_plaques"
val_x= "age"
title = val_x + " vs " + val_y


fig = px.scatter(df, x=val_x, y=val_y, trendline="ols", title=title)
#trendline="ols",
fig.update_layout(height = 700, width = 900)
fig.show()

#results = px.get_trendline_results(fig)
#print(results.sumaary())

output = sm.OLS(df[val_y], df[val_x]).fit() 
print(output.summary())

                                 OLS Regression Results                                
Dep. Variable:          total_plaques   R-squared (uncentered):                   0.475
Model:                            OLS   Adj. R-squared (uncentered):              0.458
Method:                 Least Squares   F-statistic:                              28.09
Date:                Mon, 08 May 2023   Prob (F-statistic):                    9.09e-06
Time:                        13:12:23   Log-Likelihood:                         -241.08
No. Observations:                  32   AIC:                                      484.2
Df Residuals:                      31   BIC:                                      485.6
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [75]:
df["sex"].values

array(['Male', 'Female', 'Female', 'Male', 'Female', 'Male', 'Male',
       'Female', 'Female', 'Female', 'Female', 'Male', 'Female', 'Female',
       'Male', 'Female', 'Male', 'Female', 'Female', 'Male', 'Female',
       'Female', 'Male', 'Female', 'Female', 'Female', 'Female', 'Female',
       'Female', 'Male', 'Female', 'Male'], dtype=object)

In [82]:
from plotly.subplots import make_subplots

fig = px.histogram(df, x="total_plaques", color="sex")
fig.update_layout(height = 800, width = 800)
fig.show()


## Box Plot for CDR Score

In [178]:
y =sorted(plaque_counts_merged["CDRScore"].unique())

In [179]:
y

[0.0, 0.5, 1.0, 2.0, 3.0, 4.0, 5.0, nan]

In [181]:
x_axis_title = "CDR Score"
y_axis_title = "Total plaque count"
title = "Model predicted - total plaque count vs CDR score"


fig = go.Figure()
for i in y:
  fig.add_trace(go.Box(y=plaque_counts_merged[plaque_counts_merged["CDRScore"]==i]["total_plaques"],
              boxpoints='all', # can also be outliers, or suspectedoutliers, or False
              jitter=0.3, # add some jitter for a better separation between points
              pointpos=0, # relative position of points wrt box
              name = i))
fig.update_layout(height=900, width = 1000)
fig.update_layout(
  xaxis_title_text=x_axis_title, # xaxis label
  yaxis_title_text=y_axis_title) # yaxis label
fig.update_layout(title=title)
fig.update_layout( plot_bgcolor='white')
fig.update_xaxes( mirror=True,
  ticks='outside',
  showline=True,
  linecolor='black',
  gridcolor='lightgrey'
)
fig.update_yaxes(
  mirror=True,
  ticks='outside',
  showline=True,
  linecolor='black',
  gridcolor='lightgrey'
)

fig.show()